In [1]:
# Import objects from pyomo package
from pyomo.environ import (ConcreteModel,
                           SolverFactory,
                           Constraint,
                           Expression,
                           TransformationFactory,
                           value,
                           units as pyunits)

# Import the main FlowsheetBlock from IDAES. The flowsheet block will contain the unit model
from idaes.core import FlowsheetBlock

# Import idaes logger to set output levels
import idaes.logger as idaeslog

from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)

# Todo: import Flash unit model from idaes.generic_models.unit_models
from idaes.generic_models.unit_models import Flash

In [2]:
from idaes.core.util.model_statistics import degrees_of_freedom, large_residuals_set

import pyomo.contrib.parmest.parmest as parmest

In [3]:
# Import plotting functions
import matplotlib.pyplot as plt

# Import numpy library 
import numpy as np

# Import Pandas
import pandas as pd

In [4]:
from HFCs_bmimBF4_PR import configuration

# Load data from csv
data = pd.read_csv('HFCSBF4_g.csv')

solver = SolverFactory('ipopt')
solver.options = {'tol': 1e-6}

In [5]:
def PR_model(data):
    
    m = ConcreteModel()

    m.fs = FlowsheetBlock(default={"dynamic": False})

    m.fs.properties = GenericParameterBlock(default=configuration)

    m.fs.state_block = m.fs.properties.state_block_class(
        default={"parameters": m.fs.properties,
                 "defined_state": True})

    m.fs.state_block.flow_mol.fix(1)
    m.fs.state_block.temperature.fix(298.15)
    m.fs.state_block.pressure.fix(250000)
    m.fs.state_block.mole_frac_comp["R32"].fix(0.05)
    m.fs.state_block.mole_frac_comp["R125"].fix(0.05)
    m.fs.state_block.mole_frac_comp["bmimBF4"].fix(0.90)
    
    m.fs.properties.PR_kappa_A['R32', 'R125'].fix(0.009)
    m.fs.properties.PR_kappa_A['R125', 'R32'].fix(0.007)
    m.fs.properties.PR_kappa_A['R32', 'bmimBF4'].fix(-0.02825)
    m.fs.properties.PR_kappa_A['bmimBF4', 'R32'].fix(-0.00684)
    m.fs.properties.PR_kappa_A['bmimBF4', 'R125'].fix(0.57152)
    m.fs.properties.PR_kappa_A['R125', 'bmimBF4'].fix(0.06453)
    
    # Initialize the flash unit
    m.fs.state_block.initialize(outlvl=idaeslog.CRITICAL)

    # Fix at actual temperature
    m.fs.state_block.temperature.fix(float(data["temperature"]))
#     m.fs.state_block.pressure.fix(float(data["pressure"]))
    m.fs.state_block.pressure.unfix()
    m.fs.state_block.mole_frac_comp["R125"].unfix()
    m.fs.state_block.mole_frac_comp["R32"].unfix()
    m.fs.state_block.mole_frac_comp["bmimBF4"].unfix()
#     m.fs.state_block.mole_frac_phase_comp['Vap', 'R32'].fix(float(data["y_R32"]))
#     m.fs.state_block.mole_frac_phase_comp['Vap', 'R125'].fix(float(data["y_R125"]))   
    m.fs.state_block.mole_frac_phase_comp['Liq', 'R32'].fix(float(data["x_R32"]))
    m.fs.state_block.mole_frac_phase_comp['Liq', 'R125'].fix(float(data["x_R125"])) 
    m.fs.state_block.mole_frac_phase_comp['Liq', 'bmimBF4'].fix(float(data["x_IL"])) 
    
    # Set bounds on variables to be estimated
    m.fs.properties.PR_kappa_A['R32', 'R125'].setlb(-3)
    m.fs.properties.PR_kappa_A['R32', 'R125'].setub(3)

    m.fs.properties.PR_kappa_A['R125', 'R32'].setlb(-3)
    m.fs.properties.PR_kappa_A['R125', 'R32'].setub(3)

    m.fs.properties.PR_kappa_A['R32', 'bmimBF4'].setlb(-3)
    m.fs.properties.PR_kappa_A['R32', 'bmimBF4'].setub(3)

    m.fs.properties.PR_kappa_A['bmimBF4', 'R32'].setlb(-3)
    m.fs.properties.PR_kappa_A['bmimBF4', 'R32'].setub(3)
    
    m.fs.properties.PR_kappa_A['bmimBF4', 'R125'].setlb(-3)
    m.fs.properties.PR_kappa_A['bmimBF4', 'R125'].setub(3)

    m.fs.properties.PR_kappa_A['R125', 'bmimBF4'].setlb(-3)
    m.fs.properties.PR_kappa_A['R125', 'bmimBF4'].setub(3)
    
    # Return initialized flash model
    return m

In [6]:
import pytest

test_data = {"temperature": 298.15, "pressure": 250000, "x_R32":0.179, "x_IL":0.821,  
             "x_R125":0.00001, "y_R125":0.00001, "y_R32":0.9999, "y_IL":0.0001}

m = PR_model(test_data)

DOF_initial = degrees_of_freedom(m)
print("The initial DOF is {0}".format(DOF_initial))


2022-03-07 13:24:06 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
The initial DOF is 1


In [7]:
variable_name = [
#                 "fs.properties.PR_kappa_A['R32', 'R125']",
#                 "fs.properties.PR_kappa_A['R125', 'R32']",
                "fs.properties.PR_kappa_A['R32', 'bmimBF4']",
                "fs.properties.PR_kappa_A['bmimBF4', 'R32']",
                "fs.properties.PR_kappa_A['bmimBF4', 'R125']",
                "fs.properties.PR_kappa_A['R125', 'bmimBF4']"]

In [8]:
def SSE(m, data):
    expr = ((float(data["pressure"]) - m.fs.state_block.pressure)**2)
    return expr*1e-4

# def SSE(m, data):
#     expr = ((float(data["y_R125"]) -
#              m.fs.state_block.mole_frac_phase_comp["Vap", "R125"])**2)


In [12]:
pest = parmest.Estimator(PR_model, data, variable_name, SSE, tee = True)

obj_value, parameters = pest.theta_est()

2022-03-07 13:48:30 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 13:48:37 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 13:48:44 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
Ipopt 3.13.2: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version

In [10]:
print("The SSE at the optimal solution is %0.6f" % obj_value)
print()
print("The values for the parameters are as follows:")
for k,v in parameters.items():
    print(k, "=", v)

The SSE at the optimal solution is 0.000000

The values for the parameters are as follows:
fs.properties.PR_kappa_A[R32,bmimBF4] = -0.026627816488320015
fs.properties.PR_kappa_A[bmimBF4,R32] = -0.34866830317346215
fs.properties.PR_kappa_A[bmimBF4,R125] = 1.0660831881135944
fs.properties.PR_kappa_A[R125,bmimBF4] = 0.06037071677474136


In [11]:
print(large_residuals_set(pest.ef_instance))

ComponentSet([])
